In [2]:
import pandas as pd

In [4]:
netflix_data = pd.read_csv('../data/raw/netflix_branded_titles_with_release_year.csv')

In [105]:
netflix_data.rename(columns={'title_desc':'combined_title'}, inplace=True)
netflix_data[['title', 'season']] = netflix_data['combined_title'].str.split(': Season ', expand=True)
netflix_data['title'] = netflix_data['title'].str.lower()
netflix_data['is_tv_show'] = netflix_data['season'].notnull()
netflix_data['release_year'] = netflix_data['release_year'].astype('int')

netflix_data = netflix_data[~netflix_data['combined_title'].isin(['TIGER & BUNNY 2'])]
netflix_data = netflix_data[~((netflix_data['combined_title'] == 'Catching Killers: Season 1') & (netflix_data['release_year'] == 2020))]


Manually deleted Tiger & Bunny 2, which was a duplicate in the original netflix data

Manually delted 1 of 2 Catching Killers:Season 1

In [106]:
good_file_names_movie = ['../data/interim/movies_matched.csv',
                         '../data/processed/good_batch_1.csv',
                         '../data/processed/good_batch_2.csv',
                ]
good_file_names_tv = ['../data/interim/tv_matched.csv',
                      '../data/processed/good_batch_tv_1.csv', 
                      '../data/processed/good_batch_tv_2.csv',
                      '../data/processed/good_batch_tv_3.csv',
                      ]

good_data_movies = pd.concat((pd.read_csv(file_name) for file_name in good_file_names_movie)).drop_duplicates(subset=['title'])
good_data_tv = pd.concat((pd.read_csv(file_name) for file_name in good_file_names_tv)).drop_duplicates(subset=['title'])

In [107]:
good_data_movies['tconst'] = good_data_movies['tconst'].str.replace('TV_SERIES_', "")

In [108]:
tconst_table = pd.concat([good_data_movies, good_data_tv[['tconst', 'title', 'release_year']]])

In [109]:
merged_final = netflix_data.merge(tconst_table, how='left', on=['title'])[['tconst', 'combined_title', 'release_year_x', 'is_top10', 'title', 'season', 'is_tv_show']].drop_duplicates()

Finally, Fix the movies that are actually TV shows.

In [113]:
good_data_movies_tv_convert = pd.concat((pd.read_csv(file_name) for file_name in good_file_names_movie)).drop_duplicates(subset=['title'])

In [115]:
movie_to_tv = good_data_movies_tv_convert[good_data_movies_tv_convert['tconst'].str.contains('TV_SERIES_')]

In [118]:
merged_final.loc[merged_final['title'].isin(movie_to_tv['title']), 'is_tv_show'] = True

Create Final CSV

In [120]:
#merged_final.to_csv('../data/final_data/netflix_final_data.csv', index=False)

JK discovered there were more mismatched ones based on merging to ratings file

In [ ]:
failed_merges = pd.read_csv('../data/interim/failed_merges.csv')

In [ ]:
merged_final.loc[merged_final['tconst'].isna(), 'tconst'] = failed_merges['tconst'].values.tolist()

In [ ]:
merged_final.to_csv('../data/final_data/netflix_final_data.csv', index=False)